<a href="https://colab.research.google.com/github/mmsamiei/thesis-prototype/blob/master/phase2/tokenize_distilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp /content/drive/My\ Drive/Datasets/wizard_of_wikipedia/train.json  ./train.json

In [4]:
!pip install transformers

     |████████████████████████████████| 450kB 3.5MB/s 
     |████████████████████████████████| 1.0MB 9.6MB/s 
     |████████████████████████████████| 870kB 22.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=782b16508ed52e485bf8ef6e0960592e5e08e23a8721dbfdbeb5e2ec6091b2cd
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
import torch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


In [0]:
import json

file = open('./train.json')
data = json.load(file)

In [0]:
def text_to_idslist(text):
  text = str(text)
  text = "[CLS] " + text + " [SEP]"
  return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

In [8]:
!pip install -U tqdm

     |████████████████████████████████| 61kB 2.2MB/s 
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [9]:
from tqdm.notebook import tqdm

new_dataset = []

for story in tqdm(data):
    dialog = story['dialog']
    history = text_to_idslist(story['chosen_topic'])
    for turn in dialog:
        if("checked_sentence" in turn and len(history) != 0):
            checked_sentence_tokenized = text_to_idslist(turn['checked_sentence'])
            for checked_sentence_key in turn['checked_sentence']:
                checked_sentence = turn['checked_sentence'][checked_sentence_key]
                for retrieved_passage in turn['retrieved_passages']:
                    for retrieved_passage_title in retrieved_passage:
                        for sentence in retrieved_passage[retrieved_passage_title]:
                            new_row = {}
                            if sentence != checked_sentence:
                                sentence_tokenized = text_to_idslist(sentence)
                                #history_tokened = text_to_idslist(history)
                                new_row['history'] = history.copy()
                                new_row['true_sentence'] = checked_sentence_tokenized
                                new_row['false_sentenc'] = sentence_tokenized
                                new_dataset.append(new_row)

                                
        tokened_next = text_to_idslist(turn['text'])
        if(len(history)==0):
          history.extend(tokened_next)
        else:
          history.extend(tokened_next[1:])

In [10]:
len_dataset = len(new_dataset)

print(len_dataset)

2775688


In [0]:
with open('tokenized_distillbert_train.json', 'w') as outfile:
  json.dump(new_dataset, outfile)

In [0]:
!cp /content/tokenized_distillbert_train.json /content/drive/My\ Drive/Thesis/phase-2